In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import random

In [ ]:
dataset_training = pd.read_csv(r'C:\Data\DeepLearningWorkshop\AMZN_train.csv')
dataset_training.head()

In [ ]:
C:\Data\DeepLearningWorkshop

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_data_scaled = sc.fit_transform(training_data)
training_data_scaled

In [ ]:
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_data_scaled[i-60:i, 0])
    y_train.append(training_data_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], \
                               X_train.shape[1], 1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [ ]:
seed = 1
np.random.seed(seed)
random.set_seed(seed)
model = Sequential()

In [ ]:
model.add(LSTM(units = 50, return_sequences = True, \
               input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularization
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularization
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularization
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

In [ ]:
# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
dataset_testing = pd.read_csv(r'C:\Data\DeepLearningWorkshop\AMZN_test.csv')
actual_stock_price = dataset_testing[['Open']].values
actual_stock_price 

In [ ]:
total_data = pd.concat((dataset_training['Open'], \
                        dataset_testing['Open']), axis = 0)

In [ ]:
inputs = total_data[len(total_data) \
         - len(dataset_testing) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 81):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], \
                             X_test.shape[1], 1))
predicted_stock_price = model.predict(X_test)
predicted_stock_price = \
sc.inverse_transform(predicted_stock_price)

In [ ]:
# Visualizing the results
plt.plot(actual_stock_price, color = 'green', \
         label = 'Real Amazon Stock Price',ls='--')
plt.plot(predicted_stock_price, color = 'red', \
         label = 'Predicted Amazon Stock Price',ls='-')
plt.title('Predicted Stock Price')
plt.xlabel('Time in days')
plt.ylabel('Real Stock Price')
plt.legend()
plt.show()